In [2]:
import pandas as pd
from tqdm import tqdm

directory = 'voterdata'
# demographic_name = 'VM2--MA--2022-12-31-DEMOGRAPHIC.tab'
# vote_history_name = 'VM2--MA--2022-12-31-VOTEHISTORY.tab'
voter_name = 'VM2Uniform--MA--2022-08-19.tab' # the uniform file combines both





In [3]:
# for now, we'll load the first 10k rows; eventually, use chunks to process the entire file
voter_data = pd.read_csv(directory + '/' + voter_name, sep='\t', nrows=10000)

voter_data.head()

C:\Users\Andre\AppData\Local\Temp\ipykernel_26876\259193643.py:2: DtypeWarning: Columns (19,26,27,30,50,57,58,61,117,126,189,304,325,393,406,408,409,451,454,465,697) have mixed types. Specify dtype option on import or set low_memory=False.
  voter_data = pd.read_csv(directory + '/' + voter_name, sep='\t', nrows=10000)


,SEQUENCE,LALVOTERID,Voters_Active,Voters_StateVoterID,Voters_CountyVoterID,VoterTelephones_LandlineAreaCode,VoterTelephones_Landline7Digit,VoterTelephones_LandlineFormatted,VoterTelephones_LandlineUnformatted,VoterTelephones_LandlineConfidenceCode,...,PRI_BLT_2009,PRI_BLT_2008,PRI_BLT_2007,PRI_BLT_2006,PRI_BLT_2005,PRI_BLT_2004,PRI_BLT_2003,PRI_BLT_2002,PRI_BLT_2001,PRI_BLT_2000
0,1,LALMA528324496,I,02FLS1497000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,LALMA518128944,I,07LME0362003,NaN,508.0,771-4848,(508) 771-4848,5.087715e+09,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,LALMA162304139,I,05WHY0374001,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,LALMA160666743,I,09BTY0175001,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,LALMA549110882,I,03SKE0954001,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# export to csv for easier visualization
voter_data.to_csv('voter_data_10k.csv')

In [5]:
# export columns to txt file for easier reference
with open('voter_data_columns.txt', 'w') as f:
    for col in voter_data.columns:
        f.write(col + '\n')

Preliminary notes on what columns might be useful or can be safely discarded

Useful:
- voter status (active/inactive)
- residence details (house/apt, sq ft, etc)
- state moved from
- 

Not useful:
- Telephones
- Name

Unsure:
- Should we only consider ZIPs surrounding Boston?
- Should we look at the older datasets or just 2022-12?



In [6]:
# Data needs to be verified
# Source: https://www.cityofboston.gov/images_documents/ZipCodes_tcm3-47884.pdf
# Dictionary for Boston Neighborhoods
boston_neighborhoods = {
    '02116': 'Back Bay',
    '02199': 'Back Bay',
    '02108': 'Beacon Hill',
    '02114': 'Beacon Hill',
    '02129': 'Charlestown',
    '02121': 'Dorchester',
    '02122': 'Dorchester',
    '02125': 'Dorchester',
    '02124': 'Dorchester',
    '02128': 'East Boston',
    '02115': 'Fenway/Kenmore',
    '02215': 'Fenway/Kenmore',
    '02136': 'Hyde Park',
    '02130': 'Jamaica Plain',
    '02126': 'Mattapan',
    '02109': 'North End',
    '02113': 'North End',
    '02131': 'Roslindale',
    '02119': 'Roxbury',
    '02120': 'Roxbury',
    '02127': 'South Boston',
    '02118': 'South End',
    '02132': 'West Roxbury',
}

surrounding_areas = {
    '02445': 'Brookline',
    '02446': 'Brookline',
    '02138': 'Cambridge',
    '02139': 'Cambridge',
    '02140': 'Cambridge',
    '02141': 'Cambridge',
    '02142': 'Cambridge',
    '02150': 'Chelsea',
    '02458': 'Newton',
    '02459': 'Newton',
    '02460': 'Newton',
    '02461': 'Newton',
    '02462': 'Newton',
    '02464': 'Newton',
    '02465': 'Newton',
    '02466': 'Newton',
    '02467': 'Newton',
    '02468': 'Newton',
    '02169': 'Quincy',
    '02170': 'Quincy',
    '02171': 'Quincy',
    '02269': 'Quincy',
    '02143': 'Somerville',
    '02144': 'Somerville',
    '02145': 'Somerville',
    '02149': 'Everett',
    '02151': 'Revere',
    '02472': 'Watertown',
    '02478': 'Belmont',
    '02026': 'Dedham',
    '02186': 'Milton',
}

# drop leading 0 from zip codes in dict, since that's how they're stored

boston_neighborhoods = {k.lstrip('0'): v for k, v in boston_neighborhoods.items()}
surrounding_areas = {k.lstrip('0'): v for k, v in surrounding_areas.items()}

print(boston_neighborhoods)

{'2116': 'Back Bay', '2199': 'Back Bay', '2108': 'Beacon Hill', '2114': 'Beacon Hill', '2129': 'Charlestown', '2121': 'Dorchester', '2122': 'Dorchester', '2125': 'Dorchester', '2124': 'Dorchester', '2128': 'East Boston', '2115': 'Fenway/Kenmore', '2215': 'Fenway/Kenmore', '2136': 'Hyde Park', '2130': 'Jamaica Plain', '2126': 'Mattapan', '2109': 'North End', '2113': 'North End', '2131': 'Roslindale', '2119': 'Roxbury', '2120': 'Roxbury', '2127': 'South Boston', '2118': 'South End', '2132': 'West Roxbury'}


In [11]:
# get all columns containing 'zip'

zip_columns = [col for col in voter_data.columns if 'zip' in col.lower()]
print(zip_columns)

# for each zip column, print first 30 values
for col in zip_columns:
    print(col)
    print(voter_data[col].value_counts().head(30))
    print('\n')

print(voter_data['Community_Service_District'].value_counts().head(30))


['Residence_Addresses_Zip', 'Residence_Addresses_ZipPlus4', 'Mailing_Addresses_Zip', 'Mailing_Addresses_ZipPlus4']
Residence_Addresses_Zip
Residence_Addresses_Zip
2632    3769
2601    3206
2630    2656
2668     174
2675      31
2537      20
2563      14
2660      13
2645      13
2540      10
2673      10
2664       9
2536       9
2638       9
2559       8
2649       8
2653       6
2655       6
2633       6
2639       5
2646       4
2631       3
2532       2
2644       2
2671       2
2635       2
2647       2
2648       1
Name: count, dtype: int64


Residence_Addresses_ZipPlus4
Residence_Addresses_ZipPlus4
3637.0    48
2453.0    37
2451.0    32
2300.0    31
3673.0    30
1614.0    30
2449.0    30
3666.0    29
6211.0    29
2472.0    28
2415.0    28
3621.0    28
3246.0    28
2473.0    28
1608.0    28
1626.0    27
2417.0    27
2312.0    27
1542.0    27
2409.0    27
1634.0    26
1615.0    26
3536.0    26
1810.0    26
1416.0    26
2304.0    26
2448.0    26
2455.0    25
2462.0    25
2532.0    

In [9]:
# sort columns by # of rows and remove the empty/least significant ones
voter_columns = voter_data.count().sort_values(ascending=True)

print(voter_columns)

Sewer_SubDistrict                              0
High_School_SubDistrict                        0
High_School_District                           0
Health_District                                0
Geological_Hazard_Abatement_District           0
                                           ...  
ElectionReturns_P08_Cnty_Pct_Dodd_D        10000
ElectionReturns_P08_Cnty_Pct_Clinton_D     10000
ElectionReturns_P08_Cnty_Pct_Biden_D       10000
ElectionReturns_P08_Cnty_Pct_Giuliani_R    10000
CommercialData_PropertyType                10000
Length: 771, dtype: int64


In [13]:
col = ['CommercialData_EstimatedHHIncome',
'CommercialData_EstimatedHHIncomeAmount',
'CommercialData_EstimatedAreaMedianHHIncome']

cols = col+zip_columns+['Ethnic_Description']

full_data = pd.read_csv(directory + '/' + voter_name,   sep='\t', usecols=cols)

,CommercialData_EstimatedHHIncome,CommercialData_EstimatedHHIncomeAmount,CommercialData_EstimatedAreaMedianHHIncome,Residence_Addresses_Zip,Residence_Addresses_ZipPlus4,Mailing_Addresses_Zip,Mailing_Addresses_ZipPlus4,Ethnic_Description
0,$75000-99999,$76957,$76957,2638,2550.0,2638.0,2550.0,NaN
1,NaN,NaN,NaN,2673,2556.0,2673.0,2556.0,Irish
2,$75000-99999,$83487,$83487,2536,6206.0,2536.0,6206.0,Dutch (Netherlands)
3,$75000-99999,$83487,$83487,2536,6206.0,2536.0,6206.0,English/Welsh
4,$250000+,$250000,NaN,2559,NaN,2559.0,NaN,English/Welsh
5,$75000-99999,$93268,$93268,2639,1008.0,2662.0,890.0,Italian
6,NaN,NaN,NaN,2660,NaN,2660.0,NaN,Irish
7,NaN,NaN,NaN,2660,NaN,2660.0,NaN,Irish
8,$50000-74999,$71070,$71070,2540,2804.0,2540.0,2804.0,NaN
9,$50000-74999,$73199,$73199,2660,2531.0,2660.0,2531.0,English/Welsh
